In [23]:
import requests
import json
import math
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from qgridnext import show_grid
from datetime import datetime, timedelta
import sys  
import os
import pandas_ta as ta


# Ensure the directories are in the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Data_Management')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Universe_Selection')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Signal_Generation')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Risk_Management')))

# Import the modules
from data import Data
from calculations import Calculations
from coarse import Coarse_1 as Coarse
from fine import Fine_1 as Fine
from entry_signal import Trend_Following, Mean_Reversion
from tail_risk import Stop_Loss, Take_Profit, Exit

In [24]:
symbols = ['BTCUSDT', 'ETHUSDT']
start_time = dt.datetime(2024, 1, 1)
end_time = dt.datetime(2024, 2, 1)
timeframe = '1h'
data = Data(symbols, timeframe, start_time, end_time).df
#Perform coarse analysis and filtering
coarse = Coarse()
fine = Fine()
tf = Trend_Following()
mr = Mean_Reversion()
cal = Calculations()

df = coarse.volume_flag(data, 1000)
df = coarse.sort_by_volume(df)
df = coarse.sort_by_std(df, 2, 2)
df = fine.above_ema(df, 2)
#Generate a signal (just for the sake of applying the risk management)
str_length = 10
str_mult = 3
_df = tf.supertrend_signals(df, str_length, str_mult)

_df

close  creturns      high  log_return  \
date                                                                    
2023-12-31 19:00:00 BTCUSDT  42475.23       NaN  42554.57         NaN   
                    ETHUSDT   2295.51       NaN   2297.18         NaN   
2023-12-31 20:00:00 BTCUSDT  42613.56  1.003257  42775.00    0.003251   
                    ETHUSDT   2303.72  1.003577   2306.60    0.003570   
2023-12-31 21:00:00 BTCUSDT  42581.10  1.002493  42638.41   -0.000762   
...                               ...       ...       ...         ...   
2024-01-31 16:00:00 ETHUSDT   2279.00  0.992808   2292.37   -0.003933   
2024-01-31 17:00:00 BTCUSDT  42656.07  1.004258  42690.42    0.003969   
                    ETHUSDT   2285.75  0.995748   2287.94    0.002957   
2024-01-31 18:00:00 BTCUSDT  42580.00  1.002467  42688.88   -0.001785   
                    ETHUSDT   2283.14  0.994611   2288.91   -0.001143   

                                  low      open     price   returns  \
date                                                                  
2023-12-31 19:00:00 BTCUSDT  42261.02  42283.58  42475.23       NaN   
                    ETHUSDT   2281.27   2281.87   2295.51       NaN   
2023-12-31 20:00:00 BTCUSDT  42431.65  42475.23  42613.56  0.003257   
                    ETHUSDT   2292.90   2295.52   2303.72  0.003577   
2023-12-31 21:00:00 BTCUSDT  42500.00  42613.57  42581.10 -0.000762   
...                               ...       ...       ...       ...   
2024-01-31 16:00:00 ETHUSDT   2263.57   2287.98   2279.00 -0.003925   
2024-01-31 17:00:00 BTCUSDT  42381.10  42487.12  42656.07  0.003976   
                    ETHUSDT   2269.03   2278.99   2285.75  0.002962   
2024-01-31 18:00:00 BTCUSDT  42555.46  42656.07  42580.00 -0.001783   
                    ETHUSDT   2281.88   2285.75   2283.14 -0.001142   

                                  volume  volume_in_dollars  ...  volume_rank  \
date                                                         ...                
2023-12-31 19:00:00 BTCUSDT   1271.68108       5.401495e+07  ...          2.0   
                    ETHUSDT  10771.91830       2.472705e+07  ...          1.0   
2023-12-31 20:00:00 BTCUSDT   1196.37856       5.098195e+07  ...          2.0   
                    ETHUSDT   8413.42600       1.938218e+07  ...          1.0   
2023-12-31 21:00:00 BTCUSDT    685.21980       2.917741e+07  ...          2.0   
...                                  ...                ...  ...          ...   
2024-01-31 16:00:00 ETHUSDT  23011.28190       5.244271e+07  ...          1.0   
2024-01-31 17:00:00 BTCUSDT   1398.31850       5.964677e+07  ...          2.0   
                    ETHUSDT  10362.16650       2.368532e+07  ...          1.0   
2024-01-31 18:00:00 BTCUSDT   1257.92790       5.356257e+07  ...          2.0   
                    ETHUSDT   9636.50420       2.200149e+07  ...          1.0   

                             std_values  std_rank         ema_2  above_ema  \
date                                                                         
2023-12-31 19:00:00 BTCUSDT         NaN       NaN           NaN          0   
                    ETHUSDT         NaN       NaN           NaN          0   
2023-12-31 20:00:00 BTCUSDT    0.002299       2.0  42544.395000          1   
                    ETHUSDT    0.002524       1.0   2299.615000          1   
2023-12-31 21:00:00 BTCUSDT    0.000539       2.0  42568.865000          1   
...                                 ...       ...           ...        ...   
2024-01-31 16:00:00 ETHUSDT    0.002781       1.0   2286.486103          0   
2024-01-31 17:00:00 BTCUSDT    0.002806       1.0  42637.722258          1   
                    ETHUSDT    0.002091       2.0   2285.995368          0   
2024-01-31 18:00:00 BTCUSDT    0.001262       1.0  42599.240753          0   
                    ETHUSDT    0.000808       2.0   2284.091789          0   

                             SUPERT_10_3.0  SUPERTd_10_3.0  SUPERTl_10_3.0  \
date       

## Position

To make a position columns, we have to take into account overlapping positions that might yield better risk adjusted performance if the state that we are trying to exploit continues. Example, if we are buying in a trend, and we get another signal while price pullbacks, and we don't get stopped from the first signal. If the second signal is strong enough or assuming that trend following will continue, we are better to keep holding the first signal which would not only increase performance, but this would also lower the trading costs (depeninding on the broker)

Note that we need to make the positions columns then it we would translate it to sessions (this would make the code reusable). Position column is formed by the trading signal column (should be in the calculations module)
What we want is a trade signal to be triggered then:

-> If we won't take any new trades until the current trade is closed (Basically, stay in the trade and ignore the 	new trade): 
1. Position column = ffill() trading signal column (we want to create a position whenever we are in a trade) 
2. Calculate trades, strategy returns, sessions (as usual) (note that at first, sessions would = to position)
2. Loop on the len(df):\
	Apply SL and TP\
	if trading signal != position column (trading signal == 1 and position column == 0): \
		position column = trading signal\
		position column.ffill()\
=> Note that sl and take profit would loop on every session which does not affect this process as sessions are updated on each loop which is what we need as tp and sl is applied on sessions not positions

-> If we want to take an additional trade while keeping the current open trade: 
1. Position column = cumsum() the trading signals (For example, if we would have a trading signal on one row and another trading signal on the next row, then the current positions would be = 2).
2. Calculate trades, strategy returns, sessions (as usual)
3. Apply sl and tp

-> We can Actually combine the two above:

Pseudo code:
1. Position column = cumsum() the entry signals
2. Calculate trades, strategy returns, sessions (as usual)
3. Apply the new position column using the entry and exit signals
4. Make sure to: Shift the position values and to clip the values between the max and min nbr of allowed positions

Note that we want correlated signals to be merged (trend following or mean reversion (for now)) and uncorrelated 

-> Be in a trade as long as a condition is still true (Ex: Stay in a trade as long as Supertrend is green):
1. Create a position column that is 1 if a condition is true, 0 otherwise
2. Apply sessions as usual
3. Apply take profit and stop loss as usual


In [25]:
test = _df
num_rows = len(test.index)
num_rows #This includes the row for each coin at each index

1488

In [26]:
#Generate a random signals column
np.random.seed(123)
test['signals'] = np.random.choice([0, 1], size = num_rows)
test['signals']

date                        
2023-12-31 19:00:00  BTCUSDT    0
                     ETHUSDT    1
2023-12-31 20:00:00  BTCUSDT    0
                     ETHUSDT    0
2023-12-31 21:00:00  BTCUSDT    0
                               ..
2024-01-31 16:00:00  ETHUSDT    0
2024-01-31 17:00:00  BTCUSDT    0
                     ETHUSDT    1
2024-01-31 18:00:00  BTCUSDT    1
                     ETHUSDT    0
Name: signals, Length: 1488, dtype: int32

In [27]:
series = test['signals']
position = np.where(series == 1, 1, 0)
position = pd.Series(position, index = series.index)
position = position.shift(test.index.get_level_values(1).nunique()).fillna(0) 
position

date                        
2023-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2023-12-31 20:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
2023-12-31 21:00:00  BTCUSDT    0.0
                               ... 
2024-01-31 16:00:00  ETHUSDT    1.0
2024-01-31 17:00:00  BTCUSDT    1.0
                     ETHUSDT    0.0
2024-01-31 18:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
Length: 1488, dtype: float64

In [28]:
test['position'] = position

In [29]:
test['position']

date                        
2023-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2023-12-31 20:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
2023-12-31 21:00:00  BTCUSDT    0.0
                               ... 
2024-01-31 16:00:00  ETHUSDT    1.0
2024-01-31 17:00:00  BTCUSDT    1.0
                     ETHUSDT    0.0
2024-01-31 18:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
Name: position, Length: 1488, dtype: float64

In [30]:
class Position():
    def __init__(self, df):
        """
        parametes:
            df: Stacked dataframe
        """
        self.df = df.copy()            
            

    def custom_positon(self, series):
        """
        This will take a series where a condition is met and return a series with the position
        The series can have values of either 1 (when condition is met) or other values (when condition is not met)

        It is basically a copy of the series while keeping only the values of 1.

        Assumes a stacked column (if the series has a multiindex of multicoins)
        returns the position column
        """
        position = np.where(series == 1, 1, 0)
        position = pd.Series(position, index = series.index)
        position = position.shift(test.index.get_level_values(1).nunique()).fillna(0) 
        return position

In [31]:
pos = Position(test)
test['position'] = pos.custom_positon(test['signals'])

In [32]:
test['position']

date                        
2023-12-31 19:00:00  BTCUSDT    0.0
                     ETHUSDT    0.0
2023-12-31 20:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
2023-12-31 21:00:00  BTCUSDT    0.0
                               ... 
2024-01-31 16:00:00  ETHUSDT    1.0
2024-01-31 17:00:00  BTCUSDT    1.0
                     ETHUSDT    0.0
2024-01-31 18:00:00  BTCUSDT    0.0
                     ETHUSDT    1.0
Name: position, Length: 1488, dtype: float64

Now for the position based on the trading signals

In [33]:
#Generate a random signals column
test['entry_signal'] = np.random.choice([0, 1], size = num_rows)
test['exit_signal'] = np.random.choice([0, 1], size = num_rows)

In [34]:
test['entry_signal']

date                        
2023-12-31 19:00:00  BTCUSDT    1
                     ETHUSDT    1
2023-12-31 20:00:00  BTCUSDT    1
                     ETHUSDT    0
2023-12-31 21:00:00  BTCUSDT    1
                               ..
2024-01-31 16:00:00  ETHUSDT    0
2024-01-31 17:00:00  BTCUSDT    0
                     ETHUSDT    1
2024-01-31 18:00:00  BTCUSDT    1
                     ETHUSDT    0
Name: entry_signal, Length: 1488, dtype: int32

In [35]:
len(test.unstack())

744

In [36]:
test = test.unstack()

In [37]:
index = test.index[0] 
test.loc[index, ('close', 'BTCUSDT')] #This is how to access a specific value

42475.23

In [47]:
import pandas as pd
import numpy as np

def calculate_position(df, _min = 0, _max = np.inf):
    """
    assumes a unstacked dataframe

    Calculates the 'Position' column, accumulating positions based on entry signals.

    Note that entry signals have a lower bound of 0 (without higher bound theoritacally), and exit signals have a range between 0 and 1

    Args:
        df: Pandas DataFrame with 'entry_signal' (0-1) and 'exit_signal' (boolean) columns.

    Returns:
        Pandas DataFrame with added 'Position' (float) and 'Session' (int) columns.
        Returns original dataframe if entry_signal and exit_signals columns are not found.
    """
    if _min < 0:
        raise ValueError("We can't take short positions, _min should be at least 0")
    
    if 'entry_signal' not in df.columns or 'exit_signal' not in df.columns:
        raise ValueError("Error: DataFrame must contain 'entry_signal' and 'exit_signal' columns.")
        
    for coin in df.columns.get_level_values(1).unique():
        df.loc[:, ('position', coin)] = 0.0

        for i in range(len(df)):
            if df.loc[df.index[i], ('entry_signal', coin)] > 0:
                df.loc[df.index[i], ('position', coin)] = (df.loc[df.index[i-1], ('position', coin)] if i > 0 else 0) + df.loc[df.index[i], ('entry_signal', coin)]
                # The above will add the entry to the current position

            elif df.loc[df.index[i], ('exit_signal', coin)] > 0:
                if df.loc[df.index[i-1], ('position', coin)] > 0 if i > 0 else False: # check to see if we were in a position previously
                    current_position = df.loc[df.index[i-1], ('position', coin)]
                    new_position = current_position - current_position * df.loc[df.index[i], ('exit_signal', coin)]
                    df.loc[df.index[i], ('position', coin)] = new_position 

            elif df.loc[df.index[i-1], ('position', coin)] > 0 if i > 0 else False:
                df.loc[df.index[i], ('position', coin)] = df.loc[df.index[i-1], ('position', coin)]

        df.loc[:, ('position', coin)] = df.loc[:, ('position', coin)].shift(1)
        df.loc[:, ('position', coin)] = np.clip(df.loc[:, ('position', coin)], _min, _max)
    
    return df


In [39]:
for coin in test.columns.get_level_values(1):
    test[f'position', coin ] =  test['entry_signal', coin].cumsum().shift(1).fillna(0)
    
test = test.stack(future_stack=True)
#Perform some calculations
test = cal.trades(test)
test = cal.strategy_returns(test)
test = cal.sessions(test)

C:\Users\yassi\AppData\Local\Temp\ipykernel_55660\874168162.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  test = test.stack()


In [48]:
test = calculate_position(test)

In [49]:
test.head()

close           creturns                high           \
                      BTCUSDT  ETHUSDT   BTCUSDT   ETHUSDT   BTCUSDT  ETHUSDT   
date                                                                            
2023-12-31 19:00:00  42475.23  2295.51       NaN       NaN  42554.57  2297.18   
2023-12-31 20:00:00  42613.56  2303.72  1.003257  1.003577  42775.00  2306.60   
2023-12-31 21:00:00  42581.10  2293.02  1.002493  0.998915  42638.41  2304.72   
2023-12-31 22:00:00  42330.49  2273.81  0.996592  0.990547  42586.64  2294.50   
2023-12-31 23:00:00  42399.99  2279.55  0.998229  0.993047  42399.99  2279.86   

                    log_return                 low           ...  \
                       BTCUSDT   ETHUSDT   BTCUSDT  ETHUSDT  ...   
date                                                         ...   
2023-12-31 19:00:00        NaN       NaN  42261.02  2281.27  ...   
2023-12-31 20:00:00   0.003251  0.003570  42431.65  2292.90  ...   
2023-12-31 21:00:00  -0.000762 -0.004655  42500.00  2291.20  ...   
2023-12-31 22:00:00  -0.005903 -0.008413  42230.08  2271.00  ...   
2023-12-31 23:00:00   0.001640  0.002521  42209.46  2265.24  ...   

                    log_price_return            strategy           session  \
                             BTCUSDT   ETHUSDT   BTCUSDT   ETHUSDT BTCUSDT   
date                                                                         
2023-12-31 19:00:00              NaN       NaN       NaN       NaN     0.0   
2023-12-31 20:00:00         0.003251  0.003570  0.003251  0.003570     0.0   
2023-12-31 21:00:00        -0.000762 -0.004655 -0.001524 -0.004655     1.0   
2023-12-31 22:00:00        -0.005903 -0.008413 -0.017709 -0.016826     2.0   
2023-12-31 23:00:00         0.001640  0.002521  0.006562  0.005042     3.0   

                            session_compound           overall_session_return  \
                    ETHUSDT          BTCUSDT   ETHUSDT                BTCUSDT   
date                                                                            
2023-12-31 19:00:00     0.0              NaN       NaN               0.003257   
2023-12-31 20:00:00     0.0         1.003257  1.003577               0.003257   
2023-12-31 21:00:00     1.0         0.998477  0.995355              -0.001523   
2023-12-31 22:00:00     1.0         0.982447  0.978748              -0.017553   
2023-12-31 23:00:00     2.0         1.006584  1.005055               0.006584   

                               
                      ETHUSDT  
date                           
2023-12-31 19:00:00  0.003577  
2023-12-31 20:00:00  0.003577  
2023-12-31 21:00:00 -0.021252  
2023-12-31 22:00:00 -0.021252  
2023-12-31 23:00:00  0.010801  

[5 rows x 62 columns]

Putting all together

In [ ]:
class Position():
    def __init__(self, df, _min = 0, _max = np.inf):
        """
        parametes:
            df: Stacked dataframe
            _min: Minimum multiple position size allowed
            _max: Maximum multiple position size allowed
        
        Note: _min, _max are used to clip the position size to a certain range. Only for the cumulated_position method
        """
        self.df = df.copy()     
        self._min = _min
        self._max = _max       
            

    def custom_positon(self, series):
        """
        This will take a series where a condition is met and return a series with the position
        The series can have values of either 1 (when condition is met) or other values (when condition is not met)

        It is basically a copy of the series while keeping only the values of 1.

        Assumes a stacked column (if the series has a multiindex of multicoins)
        returns the position column
        """
        position = np.where(series == 1, 1, 0)
        position = pd.Series(position, index = series.index)
        position = position.shift(test.index.get_level_values(1).nunique()).fillna(0) 
        return position


    def calculate_position(self, df, _min = 0, _max = np.inf):
        """
        assumes a unstacked dataframe

        Calculates the 'Position' column, accumulating positions based on entry signals.

        Note that entry signals have a lower bound of 0 (without higher bound theoritacally), and exit signals have a range between 0 and 1

        Args:
            df: Pandas DataFrame with 'entry_signal' (0-1) and 'exit_signal' (boolean) columns.

        Returns:
            Pandas DataFrame with added 'Position' (float) and 'Session' (int) columns.
            Returns original dataframe if entry_signal and exit_signals columns are not found.
        """
        if _min < 0:
            raise ValueError("We can't take short positions, _min should be at least 0")
        
        if 'entry_signal' not in df.columns or 'exit_signal' not in df.columns:
            raise ValueError("Error: DataFrame must contain 'entry_signal' and 'exit_signal' columns.")
            
        for coin in df.columns.get_level_values(1).unique():
            df.loc[:, ('position', coin)] = 0.0

            for i in range(len(df)):
                if df.loc[df.index[i], ('entry_signal', coin)] > 0:
                    df.loc[df.index[i], ('position', coin)] = (df.loc[df.index[i-1], ('position', coin)] if i > 0 else 0) + df.loc[df.index[i], ('entry_signal', coin)]
                    # The above will add the entry to the current position

                elif df.loc[df.index[i], ('exit_signal', coin)] > 0:
                    if df.loc[df.index[i-1], ('position', coin)] > 0 if i > 0 else False: # check to see if we were in a position previously
                        current_position = df.loc[df.index[i-1], ('position', coin)]
                        new_position = current_position - current_position * df.loc[df.index[i], ('exit_signal', coin)]
                        df.loc[df.index[i], ('position', coin)] = new_position 

                elif df.loc[df.index[i-1], ('position', coin)] > 0 if i > 0 else False:
                    df.loc[df.index[i], ('position', coin)] = df.loc[df.index[i-1], ('position', coin)]

            df.loc[:, ('position', coin)] = df.loc[:, ('position', coin)].shift(1)
            df.loc[:, ('position', coin)] = np.clip(df.loc[:, ('position', coin)], _min, _max)
        
        return df
    
    def cumulated_position(self):
        """
        
        """
        _df = self.df.unstack()
        for coin in _df.columns.get_level_values(1):
            _df[f'position', coin ] =  _df['entry_signal', coin].cumsum().shift(1).fillna(0)
            
        _df = _df.stack(future_stack=True)
        #Perform some calculations
        _df = cal.trades(_df)
        _df = cal.strategy_returns(_df)
        _df = cal.sessions(_df)

        _df = self.calculate_position(_df, self._min, self._max)

        return _df.stack(future_stack=True)
